In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [5]:
def format_df(filename):
    df = pd.read_csv(filename)
    df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
    df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
    df['date'] = df['harvest'].dt.strftime("%Y")
    df = df.rename(columns={'cname': 'Region'})
    df['gwad'] =df['gwad']/1000
    #df = df.rename(columns={df.columns[1]: "Region"})
    #df['Region'] = df['Region'].str.upper()
    return df

In [6]:
unimodal=format_df('rheas/data/tanzania_tamsat_25km_districs_unimod_dssatTable_2021_2023_100kg.csv')
bimodal = format_df('rheas/data/tanzania_tamsat_25km_districs_bimod_dssatTable_2021_2023_100kg.csv')

In [7]:
# Unimodal season:  October--August (LR)
lrStart = [9, 10, 11, 12, 1,2, 3, 4]
lrEnd = [2, 3, 4, 5, 6, 7, 8]
def LR_metrics(lr, lrStart, lrEnd, season):
    lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
    lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return lr

rh_lr = LR_metrics(unimodal, lrStart, lrEnd, "LR")
print(rh_lr.head())

     Region  date Season      gwad
0      Bahi  2022     LR  3.650338
1      Bahi  2023     LR  3.348137
2   Buhigwe  2022     LR  3.690844
3   Buhigwe  2023     LR  2.491263
4  Chamwino  2022     LR  3.763025


In [8]:
rh_lr23=rh_lr[rh_lr["date"] == '2023'] 
print(rh_lr23.head())

     Region  date Season      gwad
1      Bahi  2023     LR  3.348137
3   Buhigwe  2023     LR  2.491263
5  Chamwino  2023     LR  3.359775
7    Chemba  2023     LR  3.238725
9    Chunya  2023     LR  3.385688


In [9]:
# Bimodal season B (MAM): January--August
mamStart = [1, 2, 3, 4]
mamEnd = [5, 6, 7, 8]
def MAM_metrics(mam, mamStart, mamEnd, season):
    mam.loc[(pd.to_datetime(mam['planting']).dt.month.isin(mamStart)) & (pd.to_datetime(mam['harvest']).dt.month.isin(mamEnd)), 'Season'] = season
    mam = mam.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return mam

rh_mam = MAM_metrics(bimodal, mamStart, mamEnd, "MAM")
print(rh_mam.head())

         Region  date Season      gwad
0        Arusha  2022    MAM  2.964150
1        Arusha  2023    MAM  2.962650
2  Arusha Urban  2022    MAM  2.748675
3  Arusha Urban  2023    MAM  2.708375
4        Babati  2022    MAM  3.759150


In [10]:
rh_mam23=rh_mam[rh_mam["date"] == '2023'] 
print(rh_mam23.head())

         Region  date Season      gwad
1        Arusha  2023    MAM  2.962650
3  Arusha Urban  2023    MAM  2.708375
5        Babati  2023    MAM  3.271025
7  Babati Urban  2023    MAM  3.066100
9      Bagamoyo  2023    MAM  2.966300


In [11]:
unimodal_shp = gpd.read_file("rheas/shapefiles/gadm36_TZA_2_LongRain.shp")
bimodal_shp = gpd.read_file("rheas/shapefiles/gadm36_TZA_2_Bimodal.shp")
unimodal_shp.rename(columns={'NAME_2':'Region'}, inplace=True)
bimodal_shp.rename(columns={'NAME_2':'Region'}, inplace=True)
country_shapefile = gpd.read_file('rheas/shapefiles/gadm36_TZA_0.shp')

#shapefile["NewDist20"] = shapefile.index
#print(shapefile)
LR23 = pd.merge(unimodal_shp, rh_lr23, on="Region")
MAM23 = pd.merge(bimodal_shp, rh_mam23, on="Region")

# Setting yield 'gwad' values over water body to numpy NaN
# LR23.loc[LR23['TYPE_2'] == 'Water body', 'gwad'] = np.NaN
# MAM23.loc[MAM23['TYPE_2'] == 'Water body', 'gwad'] = np.NaN
# def exlude(dataframe):
#     if dataframe['TYPE_2'] == "Wilaya":
#         return dataframe
# exlude(MAM23)
#MAM23.to_csv('rheas/dummy.csv',index=True)


In [12]:
geo_sourceLR23 = GeoJSONDataSource(geojson=LR23.to_json())
geo_sourceMAM23 = GeoJSONDataSource(geojson=MAM23.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [13]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='Long Rains Yield (Mt/Ha) - 2023',
           outer_width=650, outer_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_sourceLR23,
          fill_color={'field': 'gwad', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('gwad', '@gwad')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [14]:
color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p2 = figure(title='MAM Yield (Mt/Ha) - 2023',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_sourceMAM23,
          fill_color={'field': 'gwad', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('gwad', '@gwad')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [15]:

p=gridplot([[p1, p2]], width=650, height=600)
show(p)

In [16]:
from bokeh.io import save
# specify the output file name and location
save(p, filename='yield_map.html', title='Yield Data Map')

C:\Users\kasit\AppData\Local\Temp\ipykernel_2104\1437694495.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(p, filename='yield_map.html', title='Yield Data Map')


'c:\\Users\\kasit\\Downloads\\MAJAMBO\\yieldprediction\\yield_map.html'

In [17]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename='yield_map.html', title='Yield Data Map')